In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from utils_analysis.Vobs_fits import Vbar_sq

In [2]:
a_0 = 1.2e-10  # MOND acceleration constant in m/s²

def load_sparc_table():
    file = "/mnt/users/koe/SPARC_Lelli2016c.mrt.txt"
    SPARC_c = ["Galaxy", "T", "D", "e_D", "f_D", "Inc",
               "e_Inc", "L", "e_L", "Reff", "SBeff", "Rdisk",
               "SBdisk", "MHI", "RHI", "Vflat", "e_Vflat", "Q", "Ref."]
    return pd.read_fwf(file, skiprows=98, names=SPARC_c)

def get_acceleration(vel, rad):
    """
    Calculate the gravitational acceleration (g_obs or g_bar).
    In the data, vel is in km/s, and rad is in kpc.
    Returns g_obs in m/s².
    """
    return (vel * 1e3)**2 / (rad * 3.086e19)  # Convert to m/s²

def superscript_number(n):
    super_digits = str.maketrans("0123456789-", "⁰¹²³⁴⁵⁶⁷⁸⁹⁻")
    return str(n).translate(super_digits)

# --- Various families of IFs ---
def n_family(n:float, g_bar):
    return ( 0.5 + np.sqrt( 1 + 4 * ( g_bar / a_0 )**(-n) ) / 2.0 )**( 1/n ) * g_bar

def delta_family(delta:float, g_bar):
    return ( 1 - np.exp( -( g_bar / a_0 )**( delta / 2.0 ) ) )**( -1.0 / delta ) * g_bar

def gamma_family(gamma:float, g_bar):
    y = g_bar / a_0
    return ( ( 1 - np.exp( -y**( gamma / 2.0 ) ) )**( -1.0 / gamma ) + ( 1 - 1.0 / gamma ) * np.exp( -y**( gamma / 2.0 ) ) ) * g_bar

# --- Specific Interpolating Functions (IFs) ---
def rar_if(g_bar):
    # return g_bar / (1 - np.exp(-np.sqrt( g_bar / a_0 )))
    return delta_family( 1.0, g_bar )

def simple_if(g_bar):
    # return g_bar/2 + np.sqrt( g_bar**2 / 4 + g_bar * a_0 )
    return n_family( 1.0, g_bar )

def standard_if(g_bar):
    # return np.sqrt( g_bar**2 / 2.0 + np.sqrt( g_bar**2 * ( g_bar**2 + 4 * a_0**2 ) ) / 2.0 )
    return n_family( 2.0, g_bar )


In [3]:
# --- Load SPARC data ---
sparc_data = {}
table = load_sparc_table()
galaxies = table["Galaxy"].values
galaxy_count = len(galaxies)

columns = [ "Rad", "Vobs", "errV", "Vgas",
            "Vdisk", "Vbul", "SBdisk", "SBbul" ]

for i in tqdm(range(galaxy_count), desc="SPARC galaxies"):
    g = table["Galaxy"][i]
    file_path = f"/mnt/users/koe/data/{g}_rotmod.dat"
    rawdata = np.loadtxt(file_path)
    data = pd.DataFrame(rawdata, columns=columns)

    bulged = np.any(data["Vbul"]>0) # Check whether galaxy has bulge.
    r = data["Rad"]

    Vobs = data["Vobs"]
    Vbar = np.sqrt( Vbar_sq(data, bulged) )
    g_obs = get_acceleration(Vobs, r)
    g_bar = get_acceleration(Vbar, r)

    sparc_data[g] = {
        'r': r,
        'Vobs': Vobs,
        'Vbar': Vbar,
        'g_obs': g_obs,
        'g_bar': g_bar,
        }

SPARC galaxies: 100%|██████████| 175/175 [00:00<00:00, 573.14it/s]


In [4]:
# Prepare fixed IF lines
x_line = np.logspace(-13, -7, 100)
fixed_if_traces = [
    go.Scatter(x=x_line, y=x_line, mode='lines', name='y=x', line=dict(color='black', dash='dash')),
    go.Scatter(x=x_line, y=simple_if(x_line), mode='lines', name='Simple IF', line=dict(color='orange')),
    go.Scatter(x=x_line, y=standard_if(x_line), mode='lines', name='Standard IF', line=dict(color='green')),
    go.Scatter(x=x_line, y=rar_if(x_line), mode='lines', name='RAR IF', line=dict(color='magenta')),
]

# Widgets
galaxy_dropdown = widgets.Dropdown(options=galaxies, description='Galaxy:')
delta_slider = widgets.FloatSlider(value=0.5, min=0.1, max=1.0, step=0.05, description='δ:')

# FigureWidget
fig = go.FigureWidget(make_subplots(
    rows=2, cols=2,
    specs=[[{"rowspan":2}, {}], [None, {}]],
    subplot_titles=("RAR", "Rotation Curves", "Acceleration Curves"),
    vertical_spacing=0.15, horizontal_spacing=0.15
))

for trace in fixed_if_traces:
    fig.add_trace(trace, row=1, col=1)

gal_traces = []

# Update function triggered automatically
def update_plot(change=None):
    global gal_traces
    # Clear previous galaxy-specific traces
    for t in gal_traces:
        fig.data = tuple(d for d in fig.data if d != t)
    gal_traces = []
    
    g = galaxy_dropdown.value
    delta_val = delta_slider.value
    vals = sparc_data[g]
    
    # Delta IF line
    delta_trace = go.Scatter(x=x_line, y=delta_family(delta_val, x_line),
                             mode='lines', line=dict(color='red', width=2), name=f'δ={delta_val:.2f}')
    fig.add_trace(delta_trace, row=1, col=1)
    gal_traces.append(delta_trace)
    
    # RAR points
    rar_trace = go.Scatter(x=vals['g_bar'], y=vals['g_obs'], mode='markers',
                           marker=dict(size=8, color='blue'), name=f'{g} RAR')
    fig.add_trace(rar_trace, row=1, col=1)
    gal_traces.append(rar_trace)
    
    # Rotation curves
    rc_obs = go.Scatter(x=vals['r'], y=vals['Vobs'], mode='lines+markers', line=dict(color='black'),
                        name=f'{g} Vobs')
    rc_bar = go.Scatter(x=vals['r'], y=vals['Vbar'], mode='lines+markers', line=dict(color='firebrick'),
                        name=f'{g} Vbar')
    fig.add_trace(rc_obs, row=1, col=2)
    fig.add_trace(rc_bar, row=1, col=2)
    gal_traces.extend([rc_obs, rc_bar])
    
    # Acceleration curves
    acc_obs = go.Scatter(x=vals['r'], y=vals['g_obs'], mode='lines+markers', line=dict(color='black'), showlegend=False)
    acc_bar = go.Scatter(x=vals['r'], y=vals['g_bar'], mode='lines+markers', line=dict(color='firebrick'), showlegend=False)
    fig.add_trace(acc_obs, row=2, col=2)
    fig.add_trace(acc_bar, row=2, col=2)
    gal_traces.extend([acc_obs, acc_bar])

# Bind update function to widget changes
galaxy_dropdown.observe(update_plot, names='value')
delta_slider.observe(update_plot, names='value')

# Display
display(widgets.VBox([galaxy_dropdown, delta_slider, fig]))

# Initialize plot
update_plot()